In [198]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [200]:
df = pd.read_csv('stud.csv')

In [202]:
df.isnull().sum()

gender                         0
race_ethnicity                 0
parental_level_of_education    0
lunch                          0
test_preparation_course        0
math_score                     0
reading_score                  0
writing_score                  0
dtype: int64

### There are no missing values

In [205]:
df.duplicated().sum()

0

### No duplicates

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [210]:
df.nunique()

gender                          2
race_ethnicity                  5
parental_level_of_education     6
lunch                           2
test_preparation_course         2
math_score                     81
reading_score                  72
writing_score                  77
dtype: int64

In [212]:
df.describe()

,math_score,reading_score,writing_score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [214]:
df.columns

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course', 'math_score', 'reading_score',
       'writing_score'],
      dtype='object')

In [216]:
numerical_features = [feature for feature in df.columns if df[feature].dtype!='O']
categorical_features = [feature for feature in df.columns if feature not in numerical_features]

print('Numerical features are :',numerical_features)
print('Categorical features are :',categorical_features)

Numerical features are : ['math_score', 'reading_score', 'writing_score']
Categorical features are : ['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']


In [218]:
for column in df.columns:
    if column in categorical_features:
        print(f'Unique values in {column} are \n{df[column].unique()}')
        print('------------------------------------------------',end="\n\n")

Unique values in gender are 
['female' 'male']
------------------------------------------------

Unique values in race_ethnicity are 
['group B' 'group C' 'group A' 'group D' 'group E']
------------------------------------------------

Unique values in parental_level_of_education are 
["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
------------------------------------------------

Unique values in lunch are 
['standard' 'free/reduced']
------------------------------------------------

Unique values in test_preparation_course are 
['none' 'completed']
------------------------------------------------



In [220]:
df['total_score'] = df['math_score']+ df['reading_score'] +  df['writing_score']
df['average'] = df['total_score']/3

In [222]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,average
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


In [224]:
math_full = df[df['math_score'] == 100]['math_score'].count()
reading_full = df[df['reading_score'] == 100]['reading_score'].count()
writing_full = df[df['writing_score'] == 100]['writing_score'].count()

print('Students with 100 in maths: ', math_full)
print('Students with 100 in reading: ', reading_full)
print('Students with 100 in writing: ', writing_full)

Students with 100 in maths:  7
Students with 100 in reading:  17
Students with 100 in writing:  14


In [226]:
math_fail = df[df['math_score'] <= 35]['math_score'].count()
reading_fail = df[df['reading_score'] <= 35]['reading_score'].count()
writing_fail = df[df['writing_score'] <= 35]['writing_score'].count()

print('Students with 100 in maths: ', math_fail)
print('Students with 100 in reading: ', reading_fail)
print('Students with 100 in writing: ', writing_fail)

Students with 100 in maths:  27
Students with 100 in reading:  15
Students with 100 in writing:  18


In [228]:
X=df.drop(columns=['total_score','math_score','average'])
y = df['math_score']

In [230]:
num_features = X.select_dtypes(exclude="O").columns
cat_features = X.select_dtypes(include="O").columns

In [232]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

one_hot = OneHotEncoder(drop='first')
std_scaler = StandardScaler()

preprocessor  = ColumnTransformer(
    [
        ("OneHotEncoder", one_hot, cat_features),
        ("StandardScaler", std_scaler, num_features)
    ]
)

In [234]:
X.shape

(1000, 7)

In [236]:
X = preprocessor.fit_transform(X)

In [238]:
X.shape

(1000, 14)

In [240]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=2)

In [244]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
def evaluate_model(true, predicted):
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    rmse = root_mean_squared_error(true, predicted)
    r_squared_score = r2_score(true, predicted)
    return mse, mae, rmse, r_squared_score

In [260]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor()
    
}

for k,v in models.items():
    model = v
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mse, model_train_mae, model_train_rmse, model_train_r2_score = evaluate_model(y_train, y_train_pred)
    model_test_mse, model_test_mae, model_test_rmse, model_test_r2_score = evaluate_model(y_test, y_test_pred)

    print(k)
    print('Model performance for training set:')
    print('- Mean squared error: ',model_train_mse)
    print('- Mean absolute error: ',model_train_mae)
    print('- Root mean squared error: ',model_train_rmse)
    print('- R2 score squared error: ',model_train_r2_score)

    print('-----------------------------------------')

    print('Model performance for test set:')
    print('- Mean squared error: ',model_test_mse)
    print('- Mean absolute error: ',model_test_mae)
    print('- Root mean squared error: ',model_test_rmse)
    print('- R2 score squared error: ',model_test_r2_score)
    print('-----------------------------------------\n')

LinearRegression
Model performance for training set:
- Mean squared error:  27.264066929277654
- Mean absolute error:  4.205477534184707
- Root mean squared error:  5.221500448077895
- R2 score squared error:  0.8831699865611634
-----------------------------------------
Model performance for test set:
- Mean squared error:  31.9082638130493
- Mean absolute error:  4.3847927256166805
- Root mean squared error:  5.648740019955715
- R2 score squared error:  0.8535008121526375
-----------------------------------------

Ridge
Model performance for training set:
- Mean squared error:  27.269423809538885
- Mean absolute error:  4.204632256539039
- Root mean squared error:  5.222013386572164
- R2 score squared error:  0.883147031644183
-----------------------------------------
Model performance for test set:
- Mean squared error:  31.916629164859508
- Mean absolute error:  4.383267694514008
- Root mean squared error:  5.64948043317786
- R2 score squared error:  0.8534624046337129
-------------